In [1]:
import sys, os
sys.path.append(os.path.abspath(".."))

print("Project path set")


Project path set


In [2]:
import pandas as pd
from src.db_connection import get_connection

conn = get_connection()

query = """
SELECT
    e.entity_id,
    e.entity_name,
    sp.provider_type
FROM entities e
LEFT JOIN entity_provider_relationships r
    ON e.entity_id = r.entity_id
LEFT JOIN service_providers sp
    ON r.provider_id = sp.provider_id
"""

df = pd.read_sql(query, conn)
conn.close()

df.head()


C:\Users\Aanchal Daryani\AppData\Local\Temp\ipykernel_32096\1900453117.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,entity_id,entity_name,provider_type
0,1,Thomas Ltd,Auditor
1,1,Thomas Ltd,Custodian
2,1,Thomas Ltd,Administrator
3,2,Dunn-Wheeler,Auditor
4,2,Dunn-Wheeler,Custodian


In [3]:
required_types = {"Auditor", "Custodian", "Administrator"}

def find_missing_providers(group):
    existing = set(group["provider_type"].dropna())
    missing = required_types - existing
    return list(missing)

missing_df = (
    df.groupby(["entity_id", "entity_name"])
      .apply(find_missing_providers)
      .reset_index(name="missing_provider_types")
)

missing_df = missing_df[missing_df["missing_provider_types"].str.len() > 0]

missing_df.head()


C:\Users\Aanchal Daryani\AppData\Local\Temp\ipykernel_32096\1088430939.py:10: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(find_missing_providers)


,entity_id,entity_name,missing_provider_types
4,5,Andrews-Hodge,[Auditor]
8,9,Ward-Russo,[Administrator]
12,13,Morgan-Arnold,[Administrator]
14,15,"Parker, Thomas and Coffey",[Custodian]
15,16,"Mcbride, Howell and Johnson",[Custodian]


In [4]:
missing_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 272 entries, 4 to 996
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   entity_id               272 non-null    int64 
 1   entity_name             272 non-null    object
 2   missing_provider_types  272 non-null    object
dtypes: int64(1), object(2)
memory usage: 8.5+ KB
